# Map GPX tracks

In [2]:
import gpxpy
import tcxparser
import pandas as pd
import numpy as np
import math
import pdb
#from folium.plugins import MiniMap
from folium import plugins
from folium.features import ColorLine
import folium
from IPython.display import display, HTML
import branca
from collections import namedtuple
import xml.etree.ElementTree as ET
import os
import matplotlib as mpl
import colorsys
from datetime import timedelta
%matplotlib inline

# Functions

In [3]:
def process_gpx_to_df(file_name):

    gpx = gpxpy.parse(open(file_name))  
    
    #(1)make DataFrame
    track = gpx.tracks[0]
    segment = track.segments[0]
    # Load the data into a Pandas dataframe (by way of a list)
    data = []
    segment_length = segment.length_3d()
    for point_idx, point in enumerate(segment.points):
        data.append([point.longitude, point.latitude,point.elevation,
                     point.time, segment.get_speed(point_idx)])
    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
    gpx_df = pd.DataFrame(data, columns=columns)
    
    #2(make points tuple for line)
    points = []
    for track in gpx.tracks:
        for segment in track.segments:        
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))
    
    return gpx_df, points

In [4]:
def process_tcx_to_df(file_name):

    tcx = tcxparser.TCXParser(file_name) 
    
    #(1)make DataFrame
    heart = tcx.hr_values()
    altitude = tcx.altitude_points()
    point = tcx.position_values()
    time = tcx.time_values()

    # Load the data into a Pandas dataframe (by way of a list)
    data = []
    for i in range(len(point)):
        data.append([point[i][1], point[i][0], altitude[i], pd.to_datetime(time[i]), heart[i]])
    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'HeartRate']
    gpx_df = pd.DataFrame(data, columns=columns)
    
    return gpx_df, point

In [5]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [6]:
def hue_gradient(mix=0):
    return mpl.colors.to_hex(colorsys.hsv_to_rgb(mix*0.8, 1, 1))

# Element tree parsing


In [7]:
def gpx_get_activity_name(gpx_file):
    #jenky - second instane of name is what I want
    root = ET.parse(gpx_file).getroot()
    for elem in root.iter():
        if elem.tag=='{http://www.topografix.com/GPX/1/1}name':
            name = elem.text
    return name

In [8]:
# return the first time point (start of the track)
def gpx_get_start(gpx_file):
    root = ET.parse(gpx_file).getroot()
    for elem in root.iter():
        if elem.tag=='{http://www.topografix.com/GPX/1/1}time':
            return pd.to_datetime(elem.text)

In [9]:
# return the first time point (start of the track)
def tcx_get_start(tcx_file):
    tcx = tcxparser.TCXParser(tcx_file)
    return pd.to_datetime(tcx.started_at)

# Main functions to build the maps

In [ ]:
def make_simple_map(file_name, map_name='my_simple_map.html', zoom_level=12):
    #convert to DF and points tuple
    df, points = process_gpx_to_df(file_name)
    print('dataframe and points created for ' + file_name)

    #get start and end lat/long
    lat_start = df.iloc[0].Latitude
    long_start = df.iloc[0].Longitude
    lat_end = df.iloc[-1].Latitude
    long_end = df.iloc[-1].Longitude

    #set activity type
    activity_color='green'
        
    #make map
    mymap = folium.Map( location=[ df.Latitude.mean(), df.Longitude.mean() ], zoom_start=zoom_level, tiles=None)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-grau/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo B&W').add_to(mymap)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo color').add_to(mymap)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.swissimage/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo satellite').add_to(mymap)
    folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
    
    # draw line
    folium.PolyLine(points, color=activity_color, weight=6, opacity=.8).add_to(mymap)
    
    folium.LayerControl(collapsed=True).add_to(mymap)
    
    mymap.save(map_name)# saves to html file for display below
    mymap

In [12]:
def make_tcx_map(file_name, map_name='my_tcx_map.html', zoom_level=12):
    #convert to DF and points tuple
    df, points = process_tcx_to_df(file_name)
    print('dataframe and points created for ' + file_name)

    #get start and end lat/long
    lat_start = df.iloc[0].Latitude
    long_start = df.iloc[0].Longitude
    lat_end = df.iloc[-1].Latitude
    long_end = df.iloc[-1].Longitude

       
    #make map
    mymap = folium.Map( location=[ df.Latitude.mean(), df.Longitude.mean() ], zoom_start=zoom_level, tiles=None)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-grau/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo B&W').add_to(mymap)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo color').add_to(mymap)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.swissimage/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo satellite').add_to(mymap)
    folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
    
    # draw line
    folium.features.ColorLine(points, list(df['HeartRate']), branca.colormap.LinearColormap(["green", "yellow", "red"], vmin=60, vmax=190), weight=6, opacity=1).add_to(mymap)
    
    folium.LayerControl(collapsed=True).add_to(mymap)
    
    mymap.save(map_name)# saves to html file for display below
    mymap

In [67]:
def map_all_tracks(gpx_folder='gpx-traces/', map_name='my_folium_all_maps.html', zoom_level=9):
   
    gpx_files = os.listdir(gpx_folder)
    gpx_files.sort(key=lambda x: os.path.getmtime(gpx_folder + x)) # to get the latest tracks plotted on top
    gpx_list = []
    tcx_list = []
    ts_min = 999999999999
    ts_max = 0
    for file in gpx_files:
            full_path_file = gpx_folder+file
            ts = 0
            if os.path.getsize(full_path_file) > 0 and full_path_file[-3:]=='gpx':
                gpx_list.append(full_path_file)
                ts = gpx_get_start(full_path_file).timestamp()
            elif os.path.getsize(full_path_file) > 0 and full_path_file[-3:]=='tcx':
                tcx_list.append(full_path_file)
                ts = tcx_get_start(full_path_file).timestamp()
            ts_min = ts if ts < ts_min else ts_min
            ts_max = ts if ts > ts_max else ts_max
    
    mymap = folium.Map( location=[ 46.8188543953265, 8.40708433025204 ], zoom_start=zoom_level, tiles=None)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-grau/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo B&W').add_to(mymap)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo color').add_to(mymap)
    folium.TileLayer('https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.swissimage/default/current/3857/{z}/{x}/{y}.jpeg', attr='&copy; <a href="https://www.swisstopo.admin.ch/">swisstopo</a>', name='Swisstopo satellite').add_to(mymap)
    folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
    
    feature_groups = {}
    for file_name in gpx_list:
        gpx = gpxpy.parse(open(file_name))
        track_name=os.path.splitext(os.path.basename(file_name))[0]
        ts = gpx_get_start(file_name).timestamp()
        #fader = hue_gradient((ts_max-ts)/(ts_max-ts_min))
        fader = hue_gradient((ts%257)/256)
        df, points = process_gpx_to_df(file_name)
        moving_data = gpx.get_moving_data(raw=True)
        elevation = gpx.get_uphill_downhill()
        print('Processed gpx for ' + file_name)
        
        # create new group for each year
        year = df.iloc[0].Time.year
        if not year in feature_groups:
            fg = folium.FeatureGroup(name=str(year), show=True)
            feature_groups[year] = fg
            mymap.add_child(fg)
        
        # add middle marker with stats
        track_summary = {}
        track_summary['Distance (km)'] = round(moving_data.moving_distance/1000,2)
        track_summary['Moving Time (hours)'] = timedelta(seconds=moving_data.moving_time)
        track_summary['Total Time (hours)'] = timedelta(seconds=gpx.get_duration())
        track_summary['Average Speed (km/h)'] = round((moving_data.moving_distance/1000)/(moving_data.moving_time/3600),2)
        track_summary['Average Speed (km/h)'] = round((moving_data.moving_distance/1000)/(gpx.get_duration()/3600),2)
        track_summary['Elevation Gain (m)'] = round(elevation.uphill)
        track_summary['Elevation Loss (m)'] = round(elevation.downhill)
        track_summary_df = pd.DataFrame(list(track_summary.items()),columns = ['A','B'])
        html_track_name = '<div align="justify"><h5>{track_name}</h5><br>{date}</div>'.format(track_name=track_name, date=gpx_get_start(file_name))
        html = html_track_name + '<div align="center">' + track_summary_df.to_html(justify='center', header=False, index=False, index_names=False, col_space=300, classes='table-condensed table-responsive table-success') + "</div>"
        popup = folium.Popup(html, max_width=300)

        #get midpoint long / lad
        length = df.shape[0]
        mid_index= math.ceil(length / 2)

        lat = df.iloc[mid_index]['Latitude']
        long = df.iloc[mid_index]['Longitude']
              
        folium.PolyLine(points, color=fader, popup=popup, weight=6, opacity=.8).add_to(mymap).add_to(feature_groups[year])

    for file_name in tcx_list:
        tcx = tcxparser.TCXParser(file_name)
        track_name=os.path.splitext(os.path.basename(file_name))[0]
        ts = tcx_get_start(file_name).timestamp()
        #fader = hue_gradient((ts_max-ts)/(ts_max-ts_min))
        fader = hue_gradient((ts%257)/256)
        df, points = process_tcx_to_df(file_name)
        moving_data = gpx.get_moving_data(raw=True)
        elevation = gpx.get_uphill_downhill()
        print('Processed tcx for ' + file_name)
        
        # create new group for each year
        year = df.iloc[0].Time.year
        if not year in feature_groups:
            fg = folium.FeatureGroup(name=str(year), show=True)
            feature_groups[year] = fg
            mymap.add_child(fg)
        
        # add middle marker with stats
        distance = 0
        for d in reversed(tcx.distance_values()):
            if d > 0:
                distance = d
                break
        track_summary = {}
        track_summary['Distance (km)'] = round(distance/1000,2)
        track_summary['Total Time (hours)'] = timedelta(seconds=tcx.duration)
        track_summary['Average Speed (km/h)'] = round((distance/1000)/(tcx.duration/3600),2)
        track_summary['Elevation Gain (m)'] = round(tcx.ascent)
        track_summary['Elevation Loss (m)'] = round(tcx.descent)
        track_summary_df = pd.DataFrame(list(track_summary.items()),columns = ['A','B'])
        html_track_name = '<div align="justify"><h5>{track_name}</h5><br>{date}</div>'.format(track_name=track_name, date=tcx_get_start(file_name))
        html = html_track_name + '<div align="center">' + track_summary_df.to_html(justify='center', header=False, index=False, index_names=False, col_space=300, classes='table-condensed table-responsive table-success') + "</div>"
        popup = folium.Popup(html, max_width=300)

        #get midpoint long / lad
        length = df.shape[0]
        mid_index= math.ceil(length / 2)

        lat = df.iloc[mid_index]['Latitude']
        long = df.iloc[mid_index]['Longitude']
        
        #folium.features.ColorLine(points, list(df['HeartRate']), branca.colormap.LinearColormap(["green", "yellow", "red"], vmin=60, vmax=190), weight=6, opacity=.8).add_to(feature_groups[year])
        folium.PolyLine(points, color=fader, popup=popup, weight=6, opacity=.8).add_to(mymap).add_to(feature_groups[year])
        
    folium.LayerControl(collapsed=True).add_to(mymap)
    mymap.save(map_name)# saves to html file

# Build all my maps

In [68]:
map_all_tracks(gpx_folder='gpx-traces/', map_name='all-tracks.html')

Processed gpx for gpx-traces/la roche devant(1).gpx
Processed gpx for gpx-traces/Montée au Devens.gpx
Processed gpx for gpx-traces/Devenchet - Gorgier.gpx
Processed gpx for gpx-traces/Mont-Aubert sur la Roche(1).gpx
Processed gpx for gpx-traces/Murist - Tour de la Molière.gpx
Processed gpx for gpx-traces/Hasliberg Panoramaweg.gpx
Processed gpx for gpx-traces/Hasliberg Mägisalp-Reuti.gpx
Processed gpx for gpx-traces/Balade au coucher du soleil.gpx
Processed gpx for gpx-traces/Gorges de Twann.gpx
Processed gpx for gpx-traces/Gorges de l'Areuse.gpx
Processed gpx for gpx-traces/Montagne du Pouce, île Maurice.gpx
Processed gpx for gpx-traces/Vaumarcus, gorges de la Vaux.gpx
Processed gpx for gpx-traces/Autour de la fruitière.gpx
Processed gpx for gpx-traces/Balade dominicale.gpx
Processed gpx for gpx-traces/La Toussuire - Le grand truc.gpx
Processed gpx for gpx-traces/La Toussuire - Plan Chaud.gpx
Processed gpx for gpx-traces/Descente du Quarteron.gpx
Processed gpx for gpx-traces/Noiraigue 

# Build map of the latest track

In [59]:
files = os.listdir('gpx-traces')
files.sort(key=lambda x: os.path.getmtime('gpx-traces/' + x))
if files[-1].endswith('.gpx'): make_simple_map('gpx-traces/' + files[-1], map_name='last-track.html')
elif files[-1].endswith('.tcx'): make_tcx_map('gpx-traces/' + files[-1], map_name='last-track.html')

dataframe and points created for gpx-traces/Gorges de Covatanne.gpx
